### Fonctions utilitaires

Compléter en ajoutant les fonctions de padding et de découpage en bloc.

In [1]:
from PIL import Image
import numpy as np
from math import log10, sqrt
import os


def load(filename):
    """
    Charge une image à partir d'un fichier et la convertit en une matrice numpy.

    Args:
        filename (str): Le chemin du fichier de l'image à charger.

    Returns:
        ndarray: Matrice représentant l'image chargée.
    """
    toLoad = Image.open(filename)
    return np.array(toLoad)


def psnr(original, compressed):
    """
    Calcule le rapport signal sur bruit (PSNR) entre deux images.

    Args:
        original (ndarray): Matrice représentant l'image originale.
        compressed (ndarray): Matrice représentant l'image compressée.

    Returns:
        float: La valeur PSNR calculée.
    """
    mse = np.mean((original.astype(int) - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


def save(matPix, filename):
    """
    Enregistre une matrice représentant une image dans un fichier.

    Args:
        matPix (ndarray): Matrice représentant l'image à enregistrer.
        filename (str): Le nom du fichier dans lequel enregistrer l'image.
    """
    Image.fromarray(matPix).save(filename)



def combler8bit(n):
    n_str = str(n)
    a_combler = 8 - len(n_str)
    nombre_comble = '0' * a_combler + n_str
    return str(nombre_comble)

### Utilitaires pour le calcul des palettes et des patchs

Fonctions auxiliaires pour manipuler les palettes, les pixels et les blocs. 

In [4]:
# Question 1

def padding(matrice):
    """
    Ajoute du remplissage (padding) noir à une image pour garantir que ses dimensions soient des multiples de 4.

    Args:
        matrice (ndarray): Matrice représentant l'image à laquelle ajouter le padding.

    Returns:
       nouvelleMat (ndarray): Matrice de l'image avec padding.
    """
    # Récupérer les dimensions de l'image
    hauteur = np.shape(matrice)[1]
    largeur = np.shape(matrice)[0]

    # Calculer le nombre de colonnes de padding nécessaires pour atteindre une dimension multiple de 4
    colonnes_supp_hauteur = (4 - hauteur % 4) % 4
    colonnes_supp_largeur = (4 - largeur % 4) % 4

    # Créer une nouvelle matrice avec les dimensions ajustées et remplie de pixels noirs
    nouvelleMat = np.zeros((largeur + colonnes_supp_largeur, hauteur + colonnes_supp_hauteur, 3), dtype = np.uint8)
    nouvelleMat[:largeur, :hauteur] = matrice

    # Enregistrer l'image avec padding
    save(nouvelleMat, "procnoir.png")

    return nouvelleMat


# Vérification de la question 1 

def no_padding(matrice, largeur, hauteur):
    """
    Supprime le padding ajouté précédemment à une image, en fonction de ses dimensions d'origine.

    Args:
        matrice (ndarray): Matrice représentant l'image avec padding.
        largeur (int): Largeur originale de l'image avant le padding.
        hauteur (int): Hauteur originale de l'image avant le padding.

    Returns:
        nouvellemat (ndarray): Matrice de l'image sans padding.
    """
    # Extraire la partie de l'image correspondant aux dimensions d'origine après le padding
    nouvellemat = matrice[:hauteur, :largeur]

    # Enregistrer l'image sans padding
    save(nouvellemat, "procsansnoir.png")

    
def verif_img_identique(image_originale, image_transformee):
    """
    Vérifie si deux images sont identiques.

    Args:
        image_originale (ndarray): Matrice représentant l'image d'origine.
        image_transformee (ndarray): Matrice représentant l'image transformée.

    Returns:
        bool: True si les deux images sont identiques, False sinon.
    """
    return np.array_equal(image_originale, image_transformee)


# Question 2 

def decouper_en_blocs(matrix):
    """
    Découpe une matrice en blocs 4x4 et les stocke dans une liste.
    
    Args:
        matrix (ndarray): Matrice à découper en blocs 4x4.
        
    Returns:
        blocs (list): Liste des blocs 4x4.
    """
    blocs = []  # Bloc final
    
    # Récupérer les dimensions de la matrice
    hauteur, largeur, canaux = matrix.shape # Canaux pour la couleur, en previsions
    
    # Parcourir la matrice en itérant sur les blocs de taille 4x4
    for i in range(0, hauteur, 4):
        for j in range(0, largeur, 4):
            # Extraire le bloc 4x4 à partir de la matrice
            bloc = matrix[i:i + 4, j:j + 4, :]
            blocs.append(bloc)  
            
    return blocs  


# Question 3

def reconstruire_image(blocs, img_originale):
    """
    Reconstruit une image à partir d'une liste de blocs 4x4.
    
    Args:
        blocs (list): Liste de blocs 4x4.
        img_originale (ndarray): Image originale à partir de laquelle les blocs ont été extraits.
        
    Returns:
        image_reconstruite (ndarray): Image reconstruite à partir de la liste blocs.
    """

    # Récupérer les dimensions de l'image originale
    hauteur, largeur, canaux = img_originale.shape
    
    # Créer une image vide avec les mêmes dimensions que l'image originale
    image_reconstruite = np.zeros((hauteur, largeur, canaux), dtype = np.uint8)
    
    # Initialise l'indice pour accéder aux blocs
    index = 0  

    # Parcourir l'image en itérant sur les blocs de taille 4x4
    for i in range(0, hauteur, 4):
        for j in range(0, largeur, 4):
            # Remplacer la section correspondante de l'image reconstruite par le bloc actuel
            image_reconstruite[i:i + 4, j:j + 4, :] = blocs[index]
            index += 1  # Passer au prochain bloc

    # Sauvegarde l'image reconstruite
    save(image_reconstruite, "imereconstruite.png")
    
    return image_reconstruite  


# Question 4

def tronque(n, p):
    """
    Enlève les p derniers bits de l'entier n.

    Args:
        n (int): Le nombre auquel on veut supprimer des bits.
        p (int): Le nombre de bit à retirer à n.

    Returns:
         n>>p : décale le nombre pour supprimer les p derniers bits.
    """
    return n>>p


def generer_palette(couleur_a, couleur_b):
    """
    Crée la palette à partir des couleurs a et b.
    
    Args:
        couleur_a (ndarray): La première couleur choisie.
        couleur_b (ndarray): La deuxième couleur choisie.
    
    Returns:
        palette (ndarray): Palette de 4 couleurs construite à partir des couleurs a et b .
    """
    # Palette de 4 couleurs avec les canaux rouge, vert et bleu

    palette = np.zeros((4, 3), dtype = np.uint8)
    
    palette[0] = couleur_a
    # Deuxième couleur de la palette : 2/3 de couleur a + 1/3 de couleur b
    palette[1] = np.round(2*couleur_a/3 +couleur_b/3)
    # Troisième couleur de la palette : 1/3 de couleur a + 2/3 de couleur b
    palette[2] = np.round(couleur_a/3 + 2*couleur_b/3)
    palette[3] = couleur_b
    
    return palette


def afficher_palette(palette):
    """
    Affiche la palette comme une image.

    Args:
        palette (ndarray): Palette à afficher.
    """
    # Créer une image avec le mode RGB et les dimensions de la palette
    img_palette = Image.new("RGB", (palette.shape[0], 1))

    # Convertir la palette en liste de tuples (R, G, B)
    palette_liste = [tuple(couleur) for couleur in palette]

    # Définir les pixels de l'image avec les couleurs de la palette
    img_palette.putdata(palette_liste)

    # Afficher l'image
    img_palette.show()


### Les méthodes de choix des couleurs de la palette


On peut essayer plusieurs stratégies pour calculer les deux couleurs a et b:
* Calculer la couleur minimale et maximale
* Prendre des couleurs moyennes (calculer moyenne et variance pour trouver les meilleurs valeurs) 
* Calculer 10 valeurs au hasard et prendre la meilleure
* Partir d'un choix et perturber pour améliorer le résultat: on teste la meilleure modification de 16 d'un des canaux d'une des couleurs

In [5]:
def trouver_couleur_proche(pixel, palette):
    """
    Trouve la couleur la plus proche .

    Args:
        pixel (ndarray): .
        palette (ndarray): .

    Returns:
        indice_couleur_proche (ndarray): .
    """
    # Initialisation de la distance minimale à une valeur supérieure à la distance maximale possible
    distance_max = np.sqrt(255 ** 2 + 255 ** 2 + 255 ** 2)
    distance_min = distance_max
    indice_couleur_proche = 0
    
    for i in range(len(palette)):
        couleur_palette = palette[i]

        pixel_entier = pixel.astype(int)
        couleur_palette_entier = couleur_palette.astype(int)
        
        differences_carrees = (pixel_entier - couleur_palette_entier) ** 2
        
        somme_carrees = np.sum(differences_carrees)
        distance = np.sqrt(somme_carrees)
        
        # Instructions print pour le débogage
        print("Pixel:", pixel_entier)
        print("Couleur de la palette:", couleur_palette_entier)
        print("Différences au carré:", differences_carrees)
        print("Distance:", distance)
        print("---")
        
        if distance < distance_min:
            distance_min = distance
            indice_couleur_proche = i
    
    print("Indice de la couleur la plus proche:", indice_couleur_proche)
        
    return indice_couleur_proche


# Trouver la couleur minimal 
def min_canal_couleur(canal_couleur: int, patch: np.ndarray)->np.uint8:
    """
    Trouve la couleur minimal du canal.

    Args:
        canal_couleur (int):
        patch (ndarray): 

    Returns:
        
    """
    assert canal_couleur <= 2 and canal_couleur >= 0
    
    minimum=np.uint8(255)
    
    for i in range(4):
        for j in range(4):
            if patch[i, j, canal_couleur] < minimum:
                minimum = patch[i, j, canal_couleur]
    
    # Mettre au bon nombre de bit

    if canal_couleur == 0:
        minimum = tronque(minimum, 8 - 5)

    elif canal_couleur == 1:
        minimum = tronque(minimum, 8 - 6)

    elif canal_couleur == 2:
        minimum = tronque(minimum, 8 - 5)

    return np.uint8(minimum)


def couleur_valeur_min(patch: np.ndarray)->np.ndarray:
    """
    Trouve la valeur de la couleur minimal.

    Args:
        patch (ndarray):.

    Returns:
        patch (ndarray): Modifie le patch donné.
    """

    return np.array([min_canal_couleur(0, patch), min_canal_couleur(1, patch), min_canal_couleur(2, patch)], dtype = np.uint8)


def max_canal_couleur(canal_couleur: int, patch: np.ndarray)->np.uint8:
    assert canal_couleur <= 2 and canal_couleur >= 0
    """
    Trouve la couleur maximal du canal.

    Args:
        canal_couleur (int): 
        patch (ndarray): 

    Returns:
        
    """
    
    maximum=np.uint8(0)
    
    for i in range(4):
        for j in range(4):
            if patch[i, j, canal_couleur] < maximum:
                maximum = patch[i, j, canal_couleur]
    
    # Mettre au bon nombre de bit

    if canal_couleur == 0:
        maximum = tronque(maximum, 8 - 5)

    elif canal_couleur == 1:
        maximum = tronque(maximum, 8 - 6)

    elif canal_couleur == 2:
        maximum = tronque(maximum, 8 - 5)
        
    return np.uint8(maximum)


def couleur_valeur_max(patch: np.ndarray)->np.ndarray:
    """
    Trouve la valeur de la couleur maximal.

    Args:
        patch (ndarray): 

    Returns:
        patch (ndarray):
    """

    return np.array([max_canal_couleur(0, patch), max_canal_couleur(1, patch), max_canal_couleur(2, patch)], dtype = np.uint8)



### Encodage d'un bloc

Écrire ici le code qui permet d'encoder un bloc et une palette en un entier.

In [20]:
def encoder_valeur_patch(couleur_a: np.ndarray, couleur_b: np.ndarray, liste_indices: list[int]) -> np.uint64:
    """
    Encode la liste des indices et la palette de couleur en un entier.

    Args:
        couleur_a (ndarray): La première couleur choisie.
        couleur_b (ndarray): La deuxième couleur choisie.
        liste_indices (list): La liste des indices, contient que des integers.

    Returns:
        patch (int): L'entier contenant les blocs et la palette                                                                                                 .
    """
    # Initialisation du patch
    patch = np.uint64(0)
    
    # Indices de pixels
    for indice in range(len(liste_indices) - 1, -1, -1):
        # laisser de l'espace
        patch <<= np.uint64(2)
        patch |= np.uint64(liste_indices[indice])
    
    # Couleur b
    for i in range(2, -1, -1):
        # laisse de l'espace
        patch <<= np.uint64(5) if i == 0 or i == 2 else np.uint64(6)
        patch |= np.uint64(couleur_b[i])
    
    # Couleur a
    for i in range(2, -1, -1):
        # laisser de l'espace
        patch <<= np.uint64(5) if i == 0 or i == 2 else np.uint64(6)
        patch |= np.uint64(couleur_a[i])
    
    return int.from_bytes(patch, byteorder = "little", signed = False)


37753843268849919


### Écriture et lecture dans un fichier

In [21]:
def compression(filepath: str, methode_min_max: bool = False):
    """
    Permet d'écrire dans le fichier les informations de l'image.

    Args:
        filepath (str): Le fichier dans lequel on veut écrire.
        methode_lin_max (bool): Permet de donner les valeurs minimales et maximales aux couleurs.

    """

    imat = load(filepath)
    imat = padding(imat)
    hauteur_image, largeur_image = np.shape(imat)[1], np.shape(imat)[0]


    nouveau_patch  = filepath.replace(filepath.split("/")[-1], filepath.split("/")[-1].split(".")[0] + ".bc1")
    if os.path.exists(nouveau_patch ) == True:
        n = 1
        nouveau_patch = nouveau_patch.replace(nouveau_patch.split("/")[-1], nouveau_patch.split("/")[-1].split(".")[0] + f"({n})" + ".bc1")
        while os.path.exists(nouveau_patch ) == True:
            n += 1
            nouveau_patch  = nouveau_patch.replace(f"({n - 1})", f"({n})")

# Compression 
    with open(nouveau_patch , "wb") as file:
        file.write("BC1".encode("utf-8"))
        if hauteur_image > 16777215 or largeur_image > 16777215:
            return "L'image est trop large pour la compression BC1"
        
        file.write(largeur_image.to_bytes(3, byteorder = "little", signed = False))
        file.write(hauteur_image.to_bytes(3, byteorder = "little", signed = False))
        bloc_image  = decouper_en_blocs(imat)
    
        for patch in bloc_image:
            if methode_min_max == True :
                couleur_a, couleur_b = couleur_valeur_min(patch), couleur_valeur_max(patch)
            palette = generer_palette(couleur_a, couleur_b)
            liste_indice = [None for k in range(16)]
            i = 0

            for l in range(4):
                for c in range(4):
                    liste_indice[i] = ((palette, patch[l, c]))
                    i += 1
            file.write(encoder_valeur_patch(couleur_a, couleur_b, liste_indice).to_bytes(8, byteorder = "little", signed = False))
    

def lire_fichier_bc1(filepath: str)->list[int]:
    """
    Lit le fichier bc1.

    Args:
        filepath (str): Le fichier que l'on veut lire, doit être un fichier bc1.

    Returns:
        hauteur (int): La hauteur de l'image.
        largeur (int): La largeur de l'image.
        liste_valeur (list[int]): La liste des valeurs .
    """
    
    with open(filepath, "rb") as file:

        if file.read(3) != "BC1".encode("utf-8"):
            return print("Ce fichier n'est pas un fichier BC1")
    
        hauteur = int.from_bytes(file.read(3), byteorder = "little", signed = False)
        largeur = int.from_bytes(file.read(3), byteorder = "little", signed = False)

        liste_valeur=[None for t in range((hauteur * largeur) // 16)]

        for i in range((hauteur * largeur) // 16):
            liste_valeur[i] = int.from_bytes(file.read(8), byteorder = "little", signed = False)
    
    return hauteur, largeur, liste_valeur


def info_patch(patch_entier: int):
    """
    Permet d'avoir les informations sur le patch.

    Args:
        patch_entier (int): .

    Returns:
        couleur_a (ndarray): La première couleur.
        couleur_b (ndarray): La deuxième couleur.
        liste_indice (liste[int]): La liste des indices.
    """

    couleur_a = [None, None, None]
    couleur_b = [None, None, None]
    liste_indice = [None for k in range(16)]

    patch_binaire = bin(patch_entier)[2:].zfill(64) 
    for i in range(2):
        for j in range(3):
            if j == 0 or j == 2: # Indice rouge ou bleu 
                if i == 0:
                    couleur_a[j] = int(patch_binaire[len(patch_binaire) - 5:], 2)
                else:
                    couleur_b[j] = int(patch_binaire[len(patch_binaire) - 5:], 2)
                patch_binaire = patch_binaire[:len(patch_binaire) - 5]

            else: # Indice vert 
                if i == 0:
                    couleur_a[j] = int(patch_binaire[len(patch_binaire) - 6:], 2)
                else:
                    couleur_b[j] = int(patch_binaire[len(patch_binaire) - 6:], 2)
                patch_binaire = patch_binaire[:len(patch_binaire) - 6]

    for i in range(16):
        liste_indice[i] = int(patch_binaire[len(patch_binaire) - 2:],2)
        patch_binaire = patch_binaire[:len(patch_binaire) -2]
    return np.array(couleur_a), np.array(couleur_b), liste_indice


def patch(couleur_a: np.ndarray, couleur_b: np.ndarray, liste_indice: list)->np.ndarray[np.ndarray[np.uint8]]:
    """
    Génère un patch.

    Args:
        couleur_a (ndarray): La première couleur choisie.
        couleur_b (ndarray): La deuxième couleur choisie.
        liste_indice (list[int]): La liste avec les indices.

    Returns:
        patch (ndarray): La liste de palette.
    """

    patch = []
    palette_couleur = generer_palette(couleur_a, couleur_b)

    ligne = []
    for index in liste_indice:
        ligne.append(palette_couleur[index])

        if len(ligne) == 4:
            patch.append(ligne)
            ligne = []
    return np.array(patch, dtype=np.uint8)

def image(filepath: str)->np.ndarray:
    """
    Prend les informations du fichier et les transforme en matrice.

    Args:
        filepath (str): Le fichier où on récupèrera les informations.

    Returns:
        nouvellemat (ndarray): Matrice de l'image sans padding.
    """

    lecture = lire_fichier_bc1(filepath)
    liste_patch = lecture[2]

    hauteur, largeur = lecture[0], lecture[1]
    couleur_a = None
    couleur_b = None
    
    liste_indice = None
    patch_liste = []

    for i in liste_patch:

        infos = info_patch(i)
        couleur_a = tronque(infos[0], 8 - 5) if i == 0 or i == 2 else tronque(infos[0], 8 - 6)
        couleur_b = tronque(infos[1], 8 - 5) if i == 0 or i == 2 else tronque(infos[1], 8 - 6)
        liste_indice = infos[2]
        
        patch_liste.append(patch(couleur_a, couleur_b, liste_indice))

    return no_padding(patch_liste, largeur, hauteur)

def decompression(filepath: str):
    """
    Décompresse une image 

    Args:
        filepath (str): Le fichier où on récupèrera les informations.

    """

    image_decompresser = image(filepath)
    nouveau_patch = filepath.replace(filepath.split("/")[-1], filepath.split("/")[-1].split(".")[0] + ".jpg")
    
    if os.path.exists(nouveau_patch) == True:
        n = 1
        nouveau_patch = nouveau_patch.replace(nouveau_patch.split("/")[-1], nouveau_patch.split("/")[-1].split(".")[0] + f"({n})" + ".jpg")
        while os.path.exists(nouveau_patch ) == True:
            n += 1
            nouveau_patch  = nouveau_patch.replace(f"({n - 1})", f"({n})")
    Image.fromarray(image_decompresser).save(nouveau_patch)

### Fonctions de test

Écrire ici tous vos tests. Chaque fonction écrite doit être testée.

In [10]:
# Section test question 1 : 
image_test = load("proc.jpg") # Ne pas utiliser celle la directement, pas multiple de 4
padding(image_test)  

imgae_bord = load("procnoir.png")
no_padding(imgae_bord, image_test.shape[1], image_test.shape[0])

# On compare les deux images 
image_reduite = load("procsansnoir.png")
identique = verif_img_identique(image_test, image_reduite)

print(f"Image avec padding sauvegardée sous le nom 'procnoir.png'")
print(f"Image réduite sauvegardée sous le nom 'procsansnoir.png'\n")
print(f"Test question 1 fonctionne :  {identique}\n")


# Section debug question 2 et 3 : 

# On décope l'image en bloc de 4
imgdecoup = decouper_en_blocs(imgae_bord)

# On la réassemble
imereconstruite = reconstruire_image(imgdecoup,imgae_bord)

print(f"Test question 2 et 3 image de base et image reconstruites sont les mêmes ?  :  {np.array_equal(imereconstruite, imgae_bord)}\n")


# Section test question 4:

# Test de la fonction tronque
n = 255  # 11111111 en binaire (8 bits)
p = 3    # On veut enlever les 3 derniers bits

print(f"test question 4 tronque on tronc 255 de 3 bit, resultat attendu, 31(11111 en binaire) : {tronque(n, p)}\n")


# Test de la fonction generer_palette
couleur_a = np.array([255, 0, 0])  # Rouge
couleur_b = np.array([0, 0, 255])  # Bleu


# Test Question 4
print(f"test question 4 palette: \n ")
print("Palette obtenue :\n", generer_palette(couleur_a, couleur_b))
afficher_palette(generer_palette(couleur_a, couleur_b))  


# Section test de la question 5 : 

# Palette de test
palette_test = generer_palette(couleur_a, couleur_b)

# Pixel de test, on prend du gris foncé
pixel_test = np.array([50, 50, 51])  

indice_couleur_proche = trouver_couleur_proche(pixel_test, palette_test)
print(indice_couleur_proche)

# Indice de la couleur de la palette la plus proche attendu pour le pixel de test
indice_couleur_attendu = 2  # Bleu, car le gris foncé est plus proche du bleu que des autes 

print(f"La question n°5 fonctionne correctement : {indice_couleur_proche == indice_couleur_attendu}")

# Section test question 6 : 

# Définition des valeurs de test pour couleur_a, couleur_b et liste_indices
couleur_a = np.array([255, 128, 64], dtype = np.uint8)
couleur_b = np.array([128, 64, 32], dtype = np.uint8)
palette = generer_palette(couleur_a, couleur_b)

# Appel de la fonction encoder_valeur_patch avec les valeurs de test
resultat_encodage = encoder_valeur_patch(couleur_a, couleur_b, [3, 2, 2, 1, 1, 3, 2, 2, 1, 2, 2, 3, 0, 1, 2, 3])
print(f"l'encodage du patch de la questio 6 est : {resultat_encodage}")

# test de trouver la couleur minimal 

image=np.array(Image.open("fin.png"))
patch = decouper_en_blocs(image)[0]
image_patch=Image.fromarray(patch)
image_patch.show()
couleur_min = couleur_valeur_min(patch)
couleur_min = np.array([[couleur_min, couleur_min],[couleur_min, couleur_min]], dtype=np.uint8)
image_min=Image.fromarray(couleur_min)
image_min.show()

# test de trouver la couleur max

image2=np.array(Image.open("fin.png"))
patch2 = decouper_en_blocs(image2)[0]
image_patch2=Image.fromarray(patch2)
image_patch2.show()
couleur_max = couleur_valeur_max(patch)
couleur_max = np.array([[couleur_max, couleur_max],[couleur_max, couleur_max]], dtype=np.uint8)
image_max=Image.fromarray(couleur_max)
image_max.show()

compression("fin.png")
compression("proc.jpg")

Image avec padding sauvegardée sous le nom 'procnoir.png'
Image réduite sauvegardée sous le nom 'procsansnoir.png'

Test question 1 fonctionne :  True

Test question 2 et 3 image de base et image reconstruites sont les mêmes ?  :  True

test question 4 tronque on tronc 255 de 3 bit, resultat attendu, 31(11111 en binaire) : 

test question 4 palette: 
 
Palette obtenue :
 [[255   0   0]
 [170   0  85]
 [ 85   0 170]
 [  0   0 255]]
Pixel: [50 50 51]
Couleur de la palette: [255   0   0]
Différences au carré: [42025  2500  2601]
Distance: 217.08523671590382
---
Pixel: [50 50 51]
Couleur de la palette: [170   0  85]
Différences au carré: [14400  2500  1156]
Distance: 134.3726162579266
---
Pixel: [50 50 51]
Couleur de la palette: [ 85   0 170]
Différences au carré: [ 1225  2500 14161]
Distance: 133.7385509118444
---
Pixel: [50 50 51]
Couleur de la palette: [  0   0 255]
Différences au carré: [ 2500  2500 41616]
Distance: 215.90738755308954
---
Indice de la couleur la plus proche: 2
2
La que